## <font color=pink>피드백<font>

<font color=pink>
1. 분석하신 결과를 더 자세히 적어주셨으면 좋았을 것 같습니다.<br>
2. 유튜브의 경우 해시태그를 빼고 크롤링해오시는 것이 정확한 분석에 더 도움이 될 것 같습니다.(해시태그만 가져올 수도 있을 것 같습니다.)<br>
3. 혹은 댓글들에서 언급되는 내용을 가져오는 것에 영상의 조회수를 가중치로 사용하는 것이 주목받는 정도를 보는데 더 나은 결과를 가져올 수도 있을 것 같습니다.<br>
4. 유튜브의 경우 조회수가 유튜브 채널의 영향을 받을 것 같아 유의미한 데이터가 될지는 잘 모르겠습니다.<br>
<font>

In [1]:
import requests
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import numpy as np

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import time
import re

In [2]:
delay = 3
browser = Chrome('/Users/philip_cho/Desktop/논문/chromedriver')
browser.implicitly_wait(delay)
browser.maximize_window() #창 화면 키우기

In [3]:
def crawl_names(keywords):
    
    candidates_data = pd.DataFrame()
    
    for keyword in keywords:
        url = 'https://www.youtube.com/results?search_query='+ keyword + '&sp=CAISBAgFEAE%253D'
        browser.get(url)
        
        browser.implicitly_wait(2)

        # 스크롤

        body = browser.find_element_by_tag_name('body')

        pages = 5
        while pages:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
            pages -= 1
            
        browser.implicitly_wait(1)

            
        # 제목, 조회수, 업로드 시간 저장
        soup = BeautifulSoup(browser.page_source,'html.parser')
        
        titles = soup.find_all('yt-formatted-string',{'class':"style-scope ytd-video-renderer","id":None})
        view_and_upload = soup.find_all('span',{'class':"style-scope ytd-video-meta-block"})

        tempViewUpload = []

        title = []
        view = []
        upload = []
        

        for i in titles:
            title.append(i.text)

        for i in view_and_upload:
            tempViewUpload.append(i.text)

        # 짝수 index = 조회수, 홀수 index = 현재 기준 업로드 일자
        for i in range(len(tempViewUpload)):
            if (i%2 == 0):
                view.append(tempViewUpload[i].split()[1])
            else:
                upload.append(tempViewUpload[i])
        try:
            candidate_data = pd.DataFrame({'title':title, 'view': view, 'upload': upload})
        except:
            print(keyword, ' 제목수:', len(title), ', view수:', len(view), ', upload수:', len(upload))
        
        candidates_data = pd.concat([candidates_data, candidate_data])
        
    return candidates_data


In [4]:
def view_eda(view):
    calc_dict = {'만회':10000, '천회':1000, '회':1, '없음':0}
    
    result = 0
    
    if view.endswith('만회'):
        result = float(view.rstrip('만회'))*calc_dict['만회']
    elif view.endswith('천회'):
        result = float(view.rstrip('천회'))*calc_dict['천회']
    elif view.endswith('회'):
        result = float(view.rstrip('회'))*calc_dict['회']
    elif view.endswith('없음'):
        result = calc_dict['만회']
        
    return result

In [5]:
keywords = ['유승민', '홍준표', '윤석열', '안철수', '최재형']

data = crawl_names(keywords)
browser.close()

In [6]:
# EDA

# reset_index
data = data.reset_index().drop('index', axis=1)

# convert views in number
for i in range(len(data.view)):
    data['view'][i] = view_eda(data['view'][i])

# count candidate from title
candidate_df = pd.DataFrame({'유승민':np.zeros(100), '홍준표':np.zeros(100), '윤석열':np.zeros(100), 
                             '안철수':np.zeros(100), '최재형':np.zeros(100)})

data = pd.concat([data,candidate_df], axis=1)

for i in range(len(data.title)):
    t = data.title[i]
    data.loc[i,'유승민'] = t.count('유승민')
    data.loc[i,'홍준표'] = t.count('홍준표')
    data.loc[i,'윤석열'] = t.count('윤석열')
    data.loc[i,'안철수'] = t.count('안철수')
    data.loc[i,'최재형'] = t.count('최재형')
    
# drop the upload time
data = data.drop('upload', axis=1)

data

,title,view,유승민,홍준표,윤석열,안철수,최재형
0,윤희숙 조수진 VS 유승민 이준석 충격 엄청남 사건...#윤희숙 #조수진 #유승민 ...,574,2.0,0.0,0.0,0.0,0.0
1,[나이트포커스] '여가부 폐지' 꺼낸 유승민·하태경...여가부 반박 / YTN,3300,1.0,0.0,0.0,0.0,0.0
2,긴급! 이준석 들이박은 윤희숙 조수진! 유승민 하태경 충격! 국민의힘 난리났다! (...,96000,1.0,0.0,0.0,0.0,0.0
3,"유승민 이준석 때문에 난리 났다,尹-安 손잡고 국힘 대권구도 출렁,대한민국 정부 미쳤다",4200,1.0,0.0,0.0,0.0,0.0
4,국민의힘 이준석대표 대구방문 #이준석 #홍준표#유승민#기자간담회,2500,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
95,[오픈베이스주가전망대응법]#오픈베이스 049480 #이루온 065440 #최재형관...,393,0.0,0.0,1.0,0.0,1.0
96,[한일단조 이루온 주가전망]최재형 정치참여 선언!!대영포장 엑세스바이오 씨젠 휴마시...,557,0.0,0.0,0.0,0.0,1.0
97,최재형 정치참여 선언! 대장주 분석 및 숨겨진 수혜주 공개 [이루온/피피아이/한일단...,259,0.0,0.0,0.0,0.0,1.0
98,"속보//포르쉐 박영수 특검 전격사퇴# 대권도전 선언 임박 최재형,진지 구축 어떻게 ...",5600,0.0,0.0,1.0,0.0,1.0


- 임시로 적은 데이터에 대해 돌려봤습니다
- 의도는 조회수와 유튜브 영상(제목)별 대권주자 언급 빈도수를 파악하여 누가 가장 유력한지(주목받고 있는지) 알아보고자 함이었습니다